Import Packages

In [ ]:
###importing all the packages required for different operations

###import to get/calculate differenet distances between eyes, lips
from scipy.spatial import distance as eu_dist
###import pretrained face landmark detections model
import dlib
###import basic image processing functions
from imutils import face_utils
from imutils.video import VideoStream
from imutils.video import FPS
import imutils
###for text to speech as alarm
import pyttsx3
###for handling different warning alarms
from threading import Thread

###other basic usage imports
import numpy as np
import argparse
import time
import cv2
import os


Warning Alarm Function

In [ ]:
###function to raise alarm for eyes closed and yawning as warning by converting text to speech

###initializing the text to speech library engine
txt_spch_engine = pyttsx3.init()

###alarm function to run in thread based enclosure
def warn_alarm(msg):
    ###globaling declaring to store the states
    global eye_close_warn_alarm
    global yawn_warn_alarm
    global bool_alarm_prsnt   ###to say if 1 alarm is already available for threading purpose

    ###using loop for eyes closed because alarm should be continuous to wake up person as person is sleeping
    while eye_close_warn_alarm:
        print("Eyes Closed Warning Alarm...")
        ###text converted to speech and played as warning alarm
        txt_spch_engine.say(msg)
        txt_spch_engine.runAndWait()

    ###not alarming continuously as person is yawning and not sleeping, he is awake 
    if yawn_warn_alarm:
        print("Yawning Warning Alarm...")
        bool_alarm_prsnt = True
        ###text converted to speech and played as warning alarm
        txt_spch_engine.say(msg)
        txt_spch_engine.runAndWait()
        bool_alarm_prsnt = False

Eye Aspect Ration(EAR) Function

In [ ]:
###function to calculate eye aspect ratio to identify whether person is sleeping or not based on eyes are closed or not 
###i.e., eye coordinate's eucledian distance using formulae 
###((sum of vertical dist)/(2*horizontal dist)) of eyes or from "EAR.jpg", (((||p2-p6||)+(||p3-p5||))/(2*(||p1-p4||))), 
###where, p1...p6 are 2d landmark locations of eye
def Single_E_A_R(side_eye):
    vert_dist1 = eu_dist.euclidean(side_eye[1], side_eye[5])   ###||p2-p6||
    vert_dist2 = eu_dist.euclidean(side_eye[2], side_eye[4])   ###||p3-p5||

    hor_dist = eu_dist.euclidean(side_eye[0], side_eye[3])   ###||p1-p4||

    sngl_ear = (vert_dist1 + vert_dist2) / (2.0 * hor_dist)   ###(((||p2-p6||)+(||p3-p5||))/(2*(||p1-p4||)))

    return sngl_ear

###calculating EAR for both left side and right side eye
def Both_Eyes_E_A_R(shape):
    ###get the coordinates, as in x,y of left and right eye from the DLIB using face_utils
    ###check coordinates of each part of face in image "Face_Landmarks.jpg", 
    ###where left eye = 36,41 and right eye = 42,47
    (left_eye_Start, left_eye_End) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]   ###for left eye("left_eye" key in dict of library)
    (right_eye_Start, right_eye_End) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]   ###for right eye("right_eye" key in dict of library)

    ###get the shapes of both eyes based on coordinates received from prev step
    leftEye_All = shape[left_eye_Start:left_eye_End]
    rightEye_All = shape[right_eye_Start:right_eye_End]

    ###calculate the EAR of both eyes from above function
    left_E_A_R = Single_E_A_R(leftEye_All)
    right_E_A_R = Single_E_A_R(rightEye_All)

    ###getting the average EAR of both eyes
    E_A_R = (left_E_A_R + right_E_A_R) / 2.0
    
    return (E_A_R, leftEye_All, rightEye_All)

Lips Distance Function

In [ ]:
###calculating the distance between upper and lower lip to see if the person is yawning or not
def Up_Low_Lip_Distance(shape):
    ###getting upper and upper-inner lip cordinates manually by checking the image "Face_Landmarks.jpg" as there is no direct key in dictionary to get individually required mouth coordinates
    up_lip = shape[50:53] 
    up_in_lip = shape[61:64]
    up_lip = np.concatenate((up_lip, up_in_lip))   ###then merge upper lip as 1

    ###getting lower and lower-inner lip cordinates manually by checking the image "Face_Landmarks.jpg" as there is no direct key in dictionary to get individually required mouth coordinates
    low_lip = shape[56:59]
    low_in_lip = shape[65:68]
    low_lip = np.concatenate((low_lip, low_in_lip))   ###then merge lower lip as 1

    ###getting avg/mean of upper and lower lip
    uplip_mean = np.mean(up_lip, axis=0)
    lowlip_mean = np.mean(low_lip, axis=0)

    ###calculating the distance between upper and lower lip
    lip_dist = abs(uplip_mean[1] - lowlip_mean[1])
    
    return lip_dist

Initializations

In [ ]:
Eye_E_A_R_Threshold = 0.30   ###mention the threshold value when the eyes to be considered as closed between 0-1 based on EAR(depends on size of eye and vary from person to person)
Eye_E_A_R_Consec_Frames = 20   ###number of frames till the eyes are closed after which the Warning Alarm should be raised 
Yawn_Threshold = 70   ###mention the threshold value for yawning as in the distance between upper and lower lip to be considered as yawning
eye_close_warn_alarm = False   ###boolean for warning alarm on closed eyes
yawn_warn_alarm = False   ###boolean for warning alarm on yawning
bool_alarm_prsnt = False   ###boolean for checking if 1 alarm is already available
Counter = 0   ###frame counter for warning alarming

Face Landmark Detector and Predictor

In [ ]:
print("Loading Face Landmark Detector and Predictor...")

###dlib's face detector using HOG + Linear SVM(slower but great accuracy)
#face_detector = dlib.get_frontal_face_detector()
###opencv's face detector using HAAR Cascades(faster but less accuracy)
face_detector = cv2.CascadeClassifier("G:\\Projects\\ADAS\\DDW_CV\\DLIB_Models\\haarcascade_frontalface_default.xml")   ###for opencv's detector use a predefined xml
###load dlib's face landmark predictor by using predtrained model 
face_predictor = dlib.shape_predictor('G:\\Projects\\ADAS\\DDW_CV\\DLIB_Models\\shape_predictor_68_face_landmarks.dat')

Video Stream for DDW

In [ ]:
print("Starting the Video Stream...")

###faster than cv2.VideoCapture and from android app webcam else give src=0 for local cam
vid_strm = VideoStream("https://192.168.0.105:8080/video").start()
time.sleep(1.0)   ###cooldown time, not cumpolsary
###save the video stream used for predictions
vid_strm_save = cv2.VideoWriter("G:\\Projects\\ADAS\\DDW_CV\\Outputs\\DDW_CV.mp4",cv2.VideoWriter_fourcc(*'MP4V'),10,(1920,1080))

###initialize the FPS counter
fps = FPS().start()

###loop through the frames from video stream
while True:
    ###get the frame from video stream
    vs_img = vid_strm.read()

    ###if no frame then exit the process
    if vs_img is None:
        break
    
    #frame = imutils.resize(frame, width=450)   ###resize for faster streaming
    gray_img = cv2.cvtColor(vs_img, cv2.COLOR_BGR2GRAY)   ###convert to grayscale for faster detection

    ###detect faces on image using the detector defined
    det_rects = face_detector.detectMultiScale(gray_img, scaleFactor=1.1, 
		minNeighbors=5, minSize=(30, 30),
		flags=cv2.CASCADE_SCALE_IMAGE)

    ###loop through the face detections
    for (x, y, w, h) in det_rects:
        ###form a dlib rectangle or box object on face in image from HAAR Cascade bounding box
        det_rect = dlib.rectangle(int(x), int(y), int(x + w),int(y + h))

        ###predict the detected face on img to get landmarks
        det_shape = face_predictor(gray_img, det_rect)
        ###convert facial landmark, i.e., x,y coordinate to numpy array
        det_shape = face_utils.shape_to_np(det_shape)

        ###get the EAR of both the eyes and each eye's coordinates
        E_A_R, leftEye_EAR, rightEye_EAR = Both_Eyes_E_A_R(det_shape)

        ###get the distance between upper and lower lips
        uplow_lip_dist = Up_Low_Lip_Distance(det_shape)

        ###to find the convexity for correction of the both eyes and draw on image for representation
        leftEye_ConvexHull = cv2.convexHull(leftEye_EAR)
        rightEye_ConvexHull = cv2.convexHull(rightEye_EAR)
        cv2.drawContours(vs_img, [leftEye_ConvexHull], -1, (0, 255, 0), 1)
        cv2.drawContours(vs_img, [rightEye_ConvexHull], -1, (0, 255, 0), 1)

        ###get the mouthcoordinates and draw them on the image for representation
        uplow_lip_outline = det_shape[48:60]
        cv2.drawContours(vs_img, [uplow_lip_outline], -1, (0, 255, 0), 1)

        ###check if the EAR is less than threshold to raise the alarm
        if E_A_R < Eye_E_A_R_Threshold :
            Counter += 1   ###increment the frame counter

            ###if the frame counter is greater than defined sleeping frames threshold, warn with alarm
            if Counter >= Eye_E_A_R_Consec_Frames :
                ###check if bool alarm is not set already as in no current current alarm 
                if eye_close_warn_alarm == False :
                    eye_close_warn_alarm = True   ###set sleeping warning bool as true
                    ###threading to handle multiple alarms at a time
                    thrd = Thread(target=warn_alarm, args=("Sleeping, Wake Up Now",))
                    thrd.deamon = True
                    thrd.start()

                ###put the drowsy info on image
                cv2.putText(vs_img, "DROWSY!!!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        else :   ###reinitialize the frame counter and alarm bool
            Counter = 0
            bool_alarm_prsnt = False

        ###check if lip distance is greater than the yawn threshold to raise alarm
        if (uplow_lip_dist > Yawn_Threshold) :
            ###check if the alarm bool is not set already as in no current current alarm
            if yawn_warn_alarm == False and bool_alarm_prsnt == False :
                yawn_warn_alarm = True   ###set yawning bool as true
                ###threading to handle multiple alarms at a time
                thrd = Thread(target=warn_alarm, args=("Yawning, Get Freshed Up",))
                thrd.deamon = True
                thrd.start()

            ###draw the yawning info on image    
            cv2.putText(vs_img, "YAWNING!!!", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
        else:   ###reinitialize the alarm bool
            yawn_warn_alarm = False

        ###put the EAR and Lip Distance on image
        cv2.putText(vs_img, "E_A_R : {:.2f}".format(E_A_R), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(vs_img, "LIPDIST : {:.2f}".format(uplow_lip_dist), (300, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    ###show the frames and save the video
    cv2.imshow("DDW_Frame", vs_img)
    vid_strm_save.write(vs_img)
    
    ###if the 'q' key was pressed, break from the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break    
    ###update the FPS counter
    fps.update()

###stop the timer and display FPS information
fps.stop()

print("Elapsed time: {:.2f}".format(fps.elapsed()))
print("Approx. FPS: {:.2f}".format(fps.fps()))

###clean the objects
cv2.destroyAllWindows()
vid_strm_save.release()
vid_strm.stop()